# Solving SAT Through Python Libraries

## Things We Are Going To Implement

1. Graph 3-coloring
   - Brute-force method
   - Backtracking (using recursion)
   - SAT solvers
2. Graph $k$-coloring (Exercise)
3. Hamiltonian Path
   - SAT solvers
   - Backtracking (Exercise)
4. Sudoku Solver (Exercise)

In [1]:
import pycosat as sat
import networkx as nx
from random import random
from itertools import product, combinations

## Graph 3-Colouring

### Creating some test graphs
1. $G_1$: An odd-cycle
2. $G_2$: Peterson Graph
3. $G_3$: A random tripartite graph
4. $G_4$: Adding a few intra-part edges to $G_3$ 

The first 3 are YES instances for 3-coloring, while the 4th is most likely a NO instance


In [2]:
G1 = nx.cycle_graph(11)

In [3]:
G2 = nx.petersen_graph()

In [10]:
# Generate a random 3-colorable graph
n = 100
p = 0.2

G3 = nx.empty_graph()

for u, v in combinations(range(n), 2):
    if (u - v) % 3 != 0 and random() < p:
        G3.add_edge(u, v)

G3.number_of_nodes(), G3.number_of_edges()

(100, 648)

In [11]:
p_in = 0.001

G4 = G3.copy()

for u, v in combinations(range(n), 2):
    if (u - v) % 3 == 0 and random() < p_in:
        G4.add_edge(u, v)

G4.number_of_nodes(), G4.number_of_edges()

(100, 649)

### Helper Functions

In [12]:
# A function to check if a given coloring is proper

def is_proper(G, col_dict):
    # Function to check if a given coloring is proper
    if G.number_of_nodes() != len(col_dict):
        return False

    for u, v in G.edges():
        if col_dict[u] == col_dict[v]:
            return False
    return True

In [ ]:
# A function to draw a graph in circular layout with nodes grouped by colors
def draw_color(G, col_dict):
    nodes_by_color = sorted(col_dict, key=col_dict.get)
    C = nx.cycle_graph(nodes_by_color)
    pos = nx.circular_layout(C)
    col_list = [col_dict[node] for node in G.nodes()]
    nx.draw_networkx(G, pos=pos, node_color=col_list,
                     with_labels=False, node_size=3000/G.number_of_nodes())